In [1]:
!nvidia-smi

Mon May 16 07:38:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    89W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
#设置路径
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! pip install -U synonyms

In [4]:
! pip install torch==1.6.0

In [5]:
! pip install transformers==4.0.1

In [6]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/usr/local/lib/python3.7/dist-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/usr/local/lib/python3.7/dist-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/usr/local/lib/python3.7/dist-packages/synonyms/data/words.vector.gz] ...


In [7]:
synonyms.nearby('篮球')

(['篮球', '橄榄球', '排球', '棒球', '美式足球', '冰球', '拳击', '网球', '高尔夫球', '高球'],
 [1.0,
  0.81482756,
  0.78554475,
  0.7845952,
  0.7815255,
  0.7550466,
  0.7411452,
  0.7350726,
  0.72586256,
  0.7199612])

# EDA(Easy Data Augmentation)
![UDA1](https://img-blog.csdnimg.cn/c5ffcca4482c4c42beb6f1215e37657c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)
### EDA：用于提高文本分类任务性能的简单数据增强技术。 EDA 由四个简单但功能强大的操作组成：同义词替换、随机插入、随机交换和随机删除。
### 之前的工作已经提出了一些 NLP 中数据增强的技术，回译通过将句子翻译成法语然后再翻译成英语来生成新数据。

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

### 停用词 stop word

停用词是指在信息检索中，为节省存储空间和提高搜索效率，在处理自然语言数据（或文本）之前或之后会自动过滤掉某些字或词，这些字或词即被称为Stop Words（停用词）
```
str = "00000003210Runoob01230000000"; 
print str.strip( '0' );  # 去除首尾字符 0

str2 = "   Runoob      ";   # 去除首尾空格
print str2.strip();

3210Runoob0123
Runoob
```


In [8]:
# strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
stop_words = {word.strip() for word in open('/content/drive/MyDrive/Colab Notebooks/dataset/baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [9]:
# 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
import random
def get_synonyms(word):
  # (['mama'],['0.9'])
  #取出元组第0个元素（['mama']），并去重
  sys = set(synonyms.nearby(word)[0])
  #将word从同义词列表中去除
  if word in sys:
    sys.remove(word)
  return list(sys)
  # 如果输入"给力" 可能没有同义词（同义词只有他自己） 则返回  ([],[])


def synonym_replacement(words, n):
  new_words = words.copy()
  # 遍历句子中的每个词，并且这个词不是停用词
  # set()去重， 以列表形式返回
  random_word_list = list(set([word for word in words if word not in stop_words]))
  # 打乱
  random.shuffle(random_word_list)
  num_replaced = 0
  for random_word in random_word_list:
    # 得到近义词列表
    synonyms = get_synonyms(random_word)
    if len(synonyms) >= 1:
      # 随机取出一个同义词
      synonym = random.choice(list(synonyms))
      # 用synonym替换原词
      new_words = [synonym if word == random_word else word for word in new_words]
      num_replaced += 1
    if num_replaced >= n:
      break
  # new_words 已经是个列表了 下一块代码举例
  sentence = ' '.join(new_words)
  new_words = sentence.split(' ')
  return new_words

### 存在一种情况  
*有近义词：**actor** -> **film star** 一个单词的近义词是两个单词*

sentence = ['in', 'actor']

*默认new_words = ['in', 'film star']*

*希望有如下表示：*
new_words = ['in', 'film', 'star']

In [10]:
# 随机删除（RD）：以概率 p 随机删除句子中的每个单词。
def random_deletion(words, p):
  if len(words) == 1:
    return words
  new_words = []
  for word in words:
    # 概率
    r = random.uniform(0, 1)
    # r>p 保留
    if r > p:
      new_words.append(word)
  # 如果都删没了 随机返回一个词
  if len(new_words) == 0:
    random_int = random.randint(0, len(words) - 1)
    return [words[random_int]]
  return new_words

In [11]:
# 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
def swap_word(new_words):
  random_idx_1 = random.randint(0, len(new_words) - 1)
  random_idx_2 = random_idx_1
  
  count = 0
  # 两者相等重新取idx_2
  while random_idx_2 == random_idx_1:
    random_idx_2 - random.randint(0, len(new_words) - 1)
    count += 1
    # 取了三次还是相等 立即推
    if count > 3:
      return new_words

  new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
  return new_words

def random_swap(words, n):
  new_words = words.copy()
  for _ in range(n):
    new_words = swap_word(new_words)
  return new_words

In [12]:
# 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
def add_word(new_words):
  # 同义词列表
  synonyms = []
  count = 0
  while len(synonyms) < 1:
    # 在句子中随机找到一个词
    random_word = new_words[random.randint(0, len(new_words) - 1)]
    synonyms = get_synonyms(random_word)
    count += 1
    # 找了10次同义词数量仍小于1，立即推
    if count >= 10:
      return 
  #将同义词表中第一个插入
  random_synonym = synonyms[0]
  # 取出要插入的位置  
  random_idx = random.randint(0, len(new_words) - 1)
  new_words.insert(random_idx, random_synonym)

def random_insert(words, n):
  new_words = words.copy()
  for i in range(n):
    add_word(new_words)
  return new_words

In [13]:
words = synonyms.seg('目前华为部分型号的手机产品出现货少的现象')
print(words)
# 后面的是词性

(['目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '少', '的', '现象'], ['t', 'nr', 'n', 'n', 'uj', 'n', 'n', 'v', 'n', 'a', 'uj', 'n'])


In [14]:
from random import shuffle
# EDA方法
# 参数num_aug：增加了几条数据
def eda(sentence, alpha_sr= 0.1, alpha_ri = 0.1, alpha_rs = 0.1, p_rd = 0.1, num_aug = 9):
  words = synonyms.seg(sentence)[0]
  num_words = len(words)
  augmented_sentence = []
  
  # 每种技术增加了多少样本
  num_new_per_tech = int(num_aug / 4) + 1
  # = 3

  # 同义词替换数量
  n_sr = max(1, int(alpha_sr * num_words))
  # 随机插入数量
  n_ri = max(1, int(alpha_ri * num_words))
  # 随机交换数量
  n_rs = max(1, int(alpha_rs * num_words))

  # 同义词替换
  for i in range(num_new_per_tech):
    a_words = synonym_replacement(words, n_sr)
    # a_words 是列表 []
    print('a-words(同义词替换):',a_words)
    augmented_sentence.append(' '.join(a_words))
  # 随机插入
  for i in range(num_new_per_tech):
    a_words = random_insert(words, n_ri)
    augmented_sentence.append(' '.join(a_words))
  # 随机交换
  for i in range(num_new_per_tech):
    a_words = random_swap(words, n_rs)
    augmented_sentence.append(' '.join(a_words))
  # 随机删除
  for i in range(num_new_per_tech):
    a_words = random_deletion(words, p_rd)
    augmented_sentence.append(' '.join(a_words))

  shuffle(augmented_sentence)

  if num_aug >= 1:
    augmented_sentence = augmented_sentence[:num_aug]
  else: # num_aug<1
    keep_prob = num_aug / len(augmented_sentence)
    # random_delete
    augmented_sentence = [s for s in augmented_sentence if random.uniform(0, 1) < keep_prob]

  return augmented_sentence



In [15]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

a-words(同义词替换): ['9', '月', '15', '日', '以来', '，', '台积电', '、', '高通', '、', '三星', '等', '华为', '的', '重要', '伙伴', '，', '只要', '没有', '美国', '的', '相关', '许可证', '，', '都', '无法', '供应', '显示卡', '给', '华为', '，', '而', '中芯国际', '等', '国产', '显示卡', '企业', '，', '也', '因', '采用', '美国', '技术', '，', '而', '无法', '供货', '给', '华为', '。', '目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '提货', '缺', '的', '现象', '，', '若', '该', '形势', '持续', '下去', '，', '华为', '手机', '销售业务', '将', '遭受重创', '摧残', '。']
a-words(同义词替换): ['9', '月', '15', '日', '以来', '，', '台积电', '、', '高通', '、', '三星', '等', '华为', '的', '重要', '供应商', '，', '只要', '没有', '美国', '的', '相关', '许可证', '，', '虽然', '无法', '市场供应', '集成电路', '给', '华为', '，', '而', '中芯国际', '等', '国产', '集成电路', '企业', '，', '也', '因', '采用', '美国', '技术', '，', '而', '无法', '发货', '给', '华为', '。', '目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '贵', '的', '现象', '，', '若', '该', '国际形势', '持续', '下去', '，', '华为', '手机', '业务', '将', '遭受', '重创', '。']
a-words(同义词替换): ['9', '月', '15', '日', '以来', '，', '台积电', '、', '高通', '、', '三星', '等', 'OPPO', '的',

['9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 显示卡 给 华为 ， 而 中芯国际 等 国产 显示卡 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 提货 缺 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 销售业务 将 遭受重创 摧残 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 华为 的 重要 供应商 ， 只要 没有 美国 的 相关 许可证 ， 虽然 无法 市场供应 集成电路 给 华为 ， 而 中芯国际 等 国产 集成电路 企业 ， 也 因 采用 美国 技术 ， 而 无法 发货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 贵 的 现象 ， 若 该 国际形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 等 OPPO 的 重要 合作伙伴 ， 只要 没有 美国 的 相关机构 许可证 ， 都 无法 供应 芯片 给 OPPO ， 而 卢戈韦 等 换代 芯片 企业 ， 也 因 采用 美国 控制技术 ， 而 无法 接单 给 OPPO 。 目前 OPPO 部分 型号 的 手机 产品 出现 装箱 少 的 现象 ， 若 该 形势 持续 下去 ， OPPO 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 台积电 、 高通 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 无法 供应 芯片 华为 ， 而 中芯国际 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 的 现

## 闭包数据增强
数据集中每条数据有两个句子 \
a, b, 1\
a, c, 1\
a, d, 0\
a-b(相似), a-c => b-c\
a-b, ad不相似 => bd不相似

In [16]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
  sentence_a = []
  sentence_b = []
  labels = []

  with open(path, 'r', encoding = 'utf8') as f:
    for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
      line = json.loads(line)
      sentence_a.append(line['sentence1'])
      sentence_b.append(line['sentence2'])
      if data_type != 'test':
        labels.append(int(line['label']))
      else:
        labels.append(0)

  df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns = ['text_a', 'text_b', 'labels'])
  return df

In [17]:
train_df = parse_data('/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json', data_type='train')

Reading train data: 100%|██████████| 34334/34334 [00:00<00:00, 97473.75it/s]


In [18]:
train_df.head()

,text_a,text_b,labels
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0
3,蚂蚁借呗多长时间综合评估一次,借呗得评估多久,0
4,我的花呗账单是***，还款怎么是***,我的花呗，月结出来说让我还***元，我自己算了一下详细名单我应该还***元,1


In [19]:
for data in train_df.groupby(by = ['text_a']):
  if len(data[1])>=3:
   
    print('数据类型:',type(data))
    print('数据长度:',len(data))
    print('data[0]:',data[0]) # text_a句子
    print('-------')
    print('data[1]:',data[1]) # 以text_a为主键返回的dataFrame
    print('-------')
  
    print('data[1].info():',data[1].info())
    break

数据类型: <class 'tuple'>
数据长度: 2
data[0]: 为什么我开通不了花呗
-------
data[1]:            text_a                            text_b  labels
21     为什么我开通不了花呗  我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以       0
1333   为什么我开通不了花呗                        为什么不可以开通花呗       0
17994  为什么我开通不了花呗                          我为何打不开花呗       0
18096  为什么我开通不了花呗                         我开通不了蚂蚁花呗       1
19484  为什么我开通不了花呗           你直接帮我查一下 确实开通不了花呗吗？我的账户       1
31930  为什么我开通不了花呗                         电脑端怎么开通花呗       0
-------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 21 to 31930
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  6 non-null      object
 1   text_b  6 non-null      object
 2   labels  6 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 192.0+ bytes
data[1].info(): None


### 举例
a : 为什么我开通不了花呗 b: 我一直想买苹果 p，没钱，想分期付款，除了花呗，还有什么可以 c:我开通不了蚂蚁花呗 ab 不相似， ac相似，=》bc不相似

In [20]:
from collections import defaultdict
def aug_group_by_a(df):
  aug_data = defaultdict(list)
  # 以text_a中的句子为g
  for g, data in df.groupby(by = ['text_a']):
    # 如果只有一条数据 无法数据增强
    if len(data) < 2:
      continue
    # iloc[ : , : ] 行列切片以“，”隔开，前面的冒号就是取行数，后面的冒号是取列数
    for i in range(len(data)):
      for j in range(i+1, len(data)):
        # 取出b的值，a,b的label
        row_i_text = data.iloc[i, 1]
        row_i_label = data.iloc[i, 2]
        # 取出c的值，a,c的label
        row_j_text = data.iloc[j, 1]
        row_j_label = data.iloc[j, 2]
        # 如果 ab ， ac都不相似则不考虑
        if row_i_label == row_j_label == 0:
          continue
        # ab相似， ac也相似， bc就相似
        aug_label = 1 if row_i_label == row_j_label == 1 else 0

        aug_data['text_a'].append(row_i_text)
        aug_data['text_b'].append(row_j_text)
        aug_data['labels'].append(aug_label)
  return pd.DataFrame(aug_data)


In [21]:
aug_train_a = aug_group_by_a(train_df)
# 看增强了多少条数据
print(aug_train_a)

                               text_a                   text_b  labels
0    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以                我开通不了蚂蚁花呗       0
1    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
2                          为什么不可以开通花呗                我开通不了蚂蚁花呗       0
3                          为什么不可以开通花呗  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
4                            我为何打不开花呗                我开通不了蚂蚁花呗       0
..                                ...                      ...     ...
223                           你是人工服务吗                   需求人工客服       0
224                           你是人工服务吗                   联系人工客服       1
225                            需求人工客服                   联系人工客服       0
226                     身份证过期可以用蚂蚁借呗吗           身份证过期可以注册蚂蚁借呗吗       0
227                            人工关闭花呗     一个帐户的花呗关，另一个帐户的花呗怎么开       0

[228 rows x 3 columns]


## UDA（Unsupervised Data Augmentation for Consistency Training）用于一致性训练的无监督数据增强
 ![UDA1](https://img-blog.csdnimg.cn/c9cb603261a1497c8093ee669b7923f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 什么是一致性训练？
### 数据增强 是 创建 逼近 真实的训练数据，并且不改变其标签
### 举例： a, b, 1  a c, 1 -> b, c, 1
### 一致性训练：增强前和增强后的标签 应该保持一致，利用这个特性训练

![unsupervised data augmentation](https://img-blog.csdnimg.cn/86f3aad1042b4bf381d662e9c3b48f0a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

 半监督学习利用无标签样本增强模型鲁棒性
 * 给定输入x, 计算输出分布$p_{\theta}(y|x)$, 同时，给输入x进行数据增强，计算出分布$p_{\theta}(y|\hat x)$.
 * 给两个分布计算KL散度

![UDA5](https://img-blog.csdnimg.cn/9d10da70d1d0467e93ef5bb1267ac87f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 解释上图
![code](https://img-blog.csdnimg.cn/d97f35fd41e0485185f40d50f4fd8e8d.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

**ps： unsup_x--(a,b)成为 no_grad_data,不需要反向传播** \
*sup:监督 unsup:无监督*

In [22]:
from bucket_sampler import SortedSampler, BucketBatchSampler
from EMA import *

In [23]:
import random
import numpy as np
import torch
config = {
        'train_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/train.json',
        'dev_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/dev.json',
        'test_file_path': '/content/drive/MyDrive/Colab Notebooks/dataset/ESIM/test.json',
        'output_path': '.',
        'model_path': '/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 400,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2022
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

In [24]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [25]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True)

  inputs['input_ids'].append(inputs_dict['input_ids'])
  inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
  inputs['attention_mask'].append(inputs_dict['attention_mask'])
  inputs['labels'].append(label)

##  对偶数据增强
sentence1: a\
sentence2: b\
a, b, 1 变成 b, a, 1

In [26]:
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
  # 左右
  lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)
  # 右左
  rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens = True, return_token_type_ids = True, return_attention_mask = True)

  # 元组的形式
  inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
  inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'], rl_inputs_dict['token_type_ids']))
  inputs['attention_mask'].append((lr_inputs_dict['attention_mask'], rl_inputs_dict['attention_mask']))
  inputs['labels'].append(label)


In [27]:
def read_data(config, tokenizer):
  train_df = parse_data(config['train_file_path'], data_type = 'train')
  dev_df = parse_data(config['dev_file_path'], data_type = 'dev')
  test_df = parse_data(config['test_file_path'], data_type = 'test')

  data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
  processed_data = {}
  unsup_data = defaultdict(list)
  for data_type, df in data_df.items():
    inputs = defaultdict(list)
    if data_type == 'train':
      reversed_inputs = defaultdict(list)

    for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total = len(df)):
      label = 0 if data_type == 'test' else row[2]
      sentence_a, sentence_b = row[0], row[1]
      build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)

      if data_type.startswith('test'):
        build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)


      build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)

    processed_data[data_type] = inputs
  processed_data['unsup_data'] = unsup_data
  return processed_data

# processed_data
# {
#    'train':,
#    'dev':,
#    'test':,
#    'unsup_data':   # 数据量最大的
# }




In [28]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:08<00:00, 445.76it/s]


In [29]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(AFQMCDataset, self).__init__()
    self.data_dict = data_dict

  def __getitem__(self, idx):
    data = (self.data_dict['input_ids'][idx],
         self.data_dict['token_type_ids'][idx],
         self.data_dict['attention_mask'][idx],
         self.data_dict['labels'][idx])
    return data
  
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [30]:
class Collator:
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
    input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype = torch.long)
    token_type_ids = torch.zeros_like(input_ids)
    attention_mask = torch.zeros_like(input_ids)
    for i in range(len(input_ids_list)):
      seq_len = len(input_ids_list[i])
      if seq_len <= max_seq_len:
        input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype = torch.long)
        token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype = torch.long)
        attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype = torch.long)

      else:
        input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id], dtype = torch.long)
        token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype = torch.long)
        attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype = torch.long)

    labels = torch.tensor(labels_list, dtype = torch.long)
    return input_ids, token_type_ids, attention_mask, labels

  def __call__(self, examples):
    input_ids_list, token_type_ids_list, attention_mask_list, labels_list = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)

    input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)

    data_dict = {
        'input_ids':input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask':attention_mask,
        'labels':labels
    }
    return data_dict

In [31]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [32]:
# UDA 无监督重新构造dataset 和 collator
class UnsupAFQMCDataset(Dataset):
  def __init__(self, data_dict):
    super(UnsupAFQMCDataset, self).__init__()
    self.data_dict = data_dict
  
  def __getitem__(self, idx):
    input_ids = self.data_dict['input_ids'][idx]
    token_type_ids = self.data_dict['token_type_ids'][idx]
    attention_mask = self.data_dict['attention_mask'][idx]
    labels = self.data_dict['labels'][idx]
    # input_ids[0]：lr_inputs_dict['input_ids']    (build_unsup_bert_inputs)
    # input_ids[1]：rl_inputs_dict['input_ids']
    return (input_ids[0], token_type_ids[0], attention_mask[0],
         input_ids[1], token_type_ids[1], attention_mask[1],
         labels)
    
  def __len__(self):
    return len(self.data_dict['input_ids'])

In [33]:
class UnsupCollator(Collator):
  def __init__(self, max_seq_len, tokenizer):
    self.max_seq_len = max_seq_len
    self.tokenizer = tokenizer

  def __call__(self, examples):
    # 根据UnsupAFQMCDataset的getitem 有七个数据
    (ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list,
     ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list,
     labels_list) = list(zip(*examples))
    cur_max_seq_len = max(len(input_ids) for input_ids in ab_input_ids_list)
    max_seq_len = min(cur_max_seq_len, self.max_seq_len)
    # 分批整ab, ba（填充与截断）
    ab_input_ids, ab_token_type_ids, ab_attention_mask, labels = self.pad_and_truncate(ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list, labels_list, max_seq_len)
    ba_input_ids, ba_token_type_ids, ba_attention_mask, labels = self.pad_and_truncate(ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list, max_seq_len)


    data_dict = {
        'ab_input_ids':ab_input_ids,
        'ab_token_type_ids':ab_token_type_ids,
        'ab_attention_mask':ab_attention_mask,
        'ba_input_ids':ba_input_ids,
        'ba_token_type_ids':ba_token_type_ids,
        'ba_attention_mask':ba_attention_mask,
        'labels':labels
    }
    return data_dict

In [34]:
from torch.utils.data import DataLoader, RandomSampler
def build_dataloader(config, data, tokenizer):
  train_dataset = AFQMCDataset(data['train'])
  dev_dataset = AFQMCDataset(data['dev'])
  test_dataset = AFQMCDataset(data['test'])
  unsup_dataset = UnsupAFQMCDataset(data['unsup_data'])

  collate_fn = Collator(config['max_seq_len'], tokenizer)
  unsup_collate_fn = UnsupCollator(config['max_seq_len'], tokenizer)

  # 使用桶采样
  if config['use_bucket']:
    # 监督数据
    # 基采样器RandomSampler
    train_sampler = RandomSampler(train_dataset)
    # drop_last 最后一个batch小于size 丢弃
    bucket_sampler = BucketBatchSampler(train_sampler, batch_size = config['batch_size'],
                      drop_last = False, sort_key = lambda x: len(train_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    train_dataloader = DataLoader(dataset = train_dataset, batch_sampler = bucket_sampler, num_workers = 4, collate_fn = collate_fn)
    # 无监督数据 
    # grad_data中(图) 有监督、无监督一起训练， 无监督数据量大， batchsize可以设置大一点
    unsup_sampler = RandomSampler(unsup_dataset)
    unsup_bucket_sampler = BucketBatchSampler(unsup_sampler, batch_size = int(config['batch_size'] * config['unsup_data_ratio']),
                      drop_last = False, sort_key = lambda x: len(unsup_dataset[x][0]),# 以 input_id 长度作为排序的指标
                      bucket_size_multiplier = config['bucket_multiplier'])
    
    unsup_dataloader = DataLoader(dataset = unsup_dataset, batch_sampler = unsup_bucket_sampler, num_workers = 4, collate_fn = unsup_collate_fn)
  # 不使用桶采样
  else:
    # 监督数据
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = config['batch_size'], shuffle = True, num_workers = 4, collate_fn = collate_fn)
    # 无监督数据
    unsup_dataloader = DataLoader(dataset = unsup_dataset, batch_size = int(config['batch_size'] * config['unsup_data_ratio']), 
                    shuffle = True, num_workers = 4, collate_fn = unsup_collate_fn)
  # 验证集、测试集dataloader 与桶采样无关 因为shuffle=false  
  dev_dataloader = DataLoader(dataset = dev_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)
  test_dataloader = DataLoader(dataset = test_dataset, batch_size = config['batch_size'], shuffle = False, num_workers = 4, collate_fn = collate_fn)

  return unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader


In [35]:
unsup_dataloader, train_dataloader, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

In [36]:
for i in train_dataloader:
  print('traindaloader中一条数据:',i)
  break

traindaloader中一条数据: {'input_ids': tensor([[ 101, 5709, 1446,  711,  784,  720, 6206, 1914, 2807, 2769,  671, 4636,
         1914, 1779,  102, 2769,  736, 3299,  819, 4638, 5709, 1446,  677, 7361,
         3221,  115,  115,  115,  711,  784,  720, 6820, 2807,  749, 2769,  115,
          115,  115, 2571, 1914,  102],
        [ 101, 6010, 6009,  955, 1446,  955, 1914,  719,  711,  784,  720, 1372,
         3300,  115,  115,  115,  702, 3299, 6821,  671,  702, 6848, 7555,  749,
          102,  711,  784,  720,  955, 1446, 1146, 3309, 1372, 5543,  115,  115,
          115,  702, 3299,  749,  102],
        [ 101, 2769,  791, 1921, 1157, 6820,  749, 5709, 1446, 2582,  720, 3766,
         4692, 1168, 7583, 2428, 2612, 1908,  102, 2582,  720, 6651, 1139,  702,
         5709, 1446,  749, 2769, 3300, 7178, 2769, 6820, 4500, 6821,  702, 8024,
         3766, 1762, 2692, 4692,  102],
        [ 101,  711,  784,  720, 2769, 4638, 3905, 2140, 5709, 1446, 1146, 3309,
         4500,  679,  749,  102,  71

In [37]:
from sklearn.metrics import f1_score, accuracy_score
def evaluation(config, model, val_dataloader):
  model.eval()
  preds = []
  labels = []
  val_loss = 0.
  val_iterator = tqdm(val_dataloader, desc = 'Evaluation', total = len(val_dataloader))

  with torch.no_grad():
    for batch in val_iterator:
      labels.append(batch['labels'])
      batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
      batch_cuda['mode'] = 'val'
      loss, logits = model(**batch_cuda)[:2]

      if config['n_gpus'] > 1:
        loss = loss.mean()

      val_loss += loss.item()
      preds.append(logits.argmax(dim = -1).detach().cpu())

  avg_val_loss = val_loss / len(val_dataloader)
  labels = torch.cat(labels, dim = 0).numpy()
  preds = torch.cat(preds, dim = 0).numpy()
  f1 = f1_score(labels, preds)
  acc = accuracy_score(labels, preds)
  return avg_val_loss, f1, acc

### 使用BertForSequenceClassification模型
(需要复写)

![BertForSequenceClassification](https://img-blog.csdnimg.cn/65a419a6c58a4b07883a3c91084e7cde.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![BertForSequenceClassification1](https://img-blog.csdnimg.cn/0c2a5f0612aa4e42a1dfdf0e0352e0f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

## X.view
```
a = torch.randn(3,5,2)
print(a)
print(a.view(-1))
```
执行结果：
```
tensor([[[-0.6887,  0.2203],
         [-1.6103, -0.7423],
         [ 0.3097, -2.9694],
         [ 1.2073, -0.3370],
         [-0.5506,  0.4753]],

        [[-1.3605,  1.9303],
         [-1.5382, -1.0865],
         [-0.9208, -0.1754],
         [ 0.1476, -0.8866],
         [ 0.4519,  0.2771]],

        [[ 0.6662,  1.1027],
         [-0.0912, -0.6284],
         [-1.0253, -0.3542],
         [ 0.6909, -1.3905],
         [-2.1140,  1.3426]]])
tensor([-0.6887,  0.2203, -1.6103, -0.7423,  0.3097, -2.9694,  1.2073, -0.3370,
        -0.5506,  0.4753, -1.3605,  1.9303, -1.5382, -1.0865, -0.9208, -0.1754,
         0.1476, -0.8866,  0.4519,  0.2771,  0.6662,  1.1027, -0.0912, -0.6284,
        -1.0253, -0.3542,  0.6909, -1.3905, -2.1140,  1.3426])
```
X.view(-1)中的-1本意是根据另外一个数来自动调整维度，但是这里只有一个维度，因此就会将X里面的所有维度数据转化成一维的，并且按先后顺序排列。

In [38]:
from transformers import BertForSequenceClassification
import torch.nn as nn
class BertForAFQMC(BertForSequenceClassification):
   # 复写forward
   def forward(self, input_ids, token_type_ids, attention_mask, labels = None, mode= 'train'):

     outputs = self.bert(input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, output_hidden_states = True)
     # 维度：[batch_size, hidden_size]
     pooled_output = outputs[1]
     pooled_output = self.dropout(pooled_output)

     logits = self.classifier(pooled_output)
     # print('BertForAFQMC中logits：',logits)
     outputs = (logits, )
     # print('BertForAFQMC中outputs：',outputs)

     if mode == 'val':
       loss_fct = nn.CrossEntropyLoss()
       # X.view(-1)中的-1本意是根据另外一个数来自动调整维度
       loss = loss_fct(logits, labels.view(-1))

       outputs = (loss, ) + outputs
     return outputs

一组中：
```
BertForAFQMC中logits： tensor([[-0.8202,  0.0187],
        [-0.7462,  0.2740],
        [-0.8858,  0.3251],
        [-0.5850, -0.0393],
        [ 0.1496,  0.2556],
        [-0.9169, -0.1298],
        [-0.8008,  0.0615],
        [-0.7671,  0.1412],
        [-1.0368,  0.1229],
        [-0.7881,  0.0400],
        [-0.6721, -0.3433],
        [-1.3198, -0.2528],
        [-0.4168,  0.2610],
        [-1.0321,  0.5436],
        [-0.9334, -0.1720],
        [-0.7633, -0.2436],
        [-1.0511,  0.0330],
        [-0.6819, -0.0422],
        [-0.7017,  0.3001],
        [-0.6800, -0.2233],
        [-0.6387,  0.0239],
        [-0.8804, -0.3109],
        [-0.9353, -0.0015],
        [-0.7683, -0.2950]], device='cuda:0')
```
```
BertForAFQMC中outputs： (tensor([[-0.8202,  0.0187],
        [-0.7462,  0.2740],
        [-0.8858,  0.3251],
        [-0.5850, -0.0393],
        [ 0.1496,  0.2556],
        [-0.9169, -0.1298],
        [-0.8008,  0.0615],
        [-0.7671,  0.1412],
        [-1.0368,  0.1229],
        [-0.7881,  0.0400],
        [-0.6721, -0.3433],
        [-1.3198, -0.2528],
        [-0.4168,  0.2610],
        [-1.0321,  0.5436],
        [-0.9334, -0.1720],
        [-0.7633, -0.2436],
        [-1.0511,  0.0330],
        [-0.6819, -0.0422],
        [-0.7017,  0.3001],
        [-0.6800, -0.2233],
        [-0.6387,  0.0239],
        [-0.8804, -0.3109],
        [-0.9353, -0.0015],
        [-0.7683, -0.2950]], device='cuda:0'),)
```

####----------------------------------------------------------------------------------

In [39]:
def get_tsa_threshold(total_steps, global_steps):
    return np.exp((global_steps / total_steps - 1) * 5) / 2 + 0.5

In [40]:
# 返回 grad_data：需要计算梯度，需要进行反向传播的数据
# 返回 no_grad_data: 不需要计算梯度，不需要进行反向传播的数据
def get_data(sup_batch, unsup_batch, config):
    grad_data = {}
    no_grad_data = {}
    # sup_batch [bs, seq_len]
    # unsup_batch [bs, seq_len]
    # 监督数据的 最长 长度
    sup_max_len = sup_batch['input_ids'].size(1)

    # 无监督数据 的最长 长度
    unsup_max_len = unsup_batch['ba_input_ids'].size(1)

    # 当前数据 的最长 长度
    cur_max_len = max(sup_max_len, unsup_max_len)

    for item, sup_value in sup_batch.items():
        if item == 'labels':
            grad_data[item] = sup_value.to(config['device'])
            continue
        
        ba_unsup_value = unsup_batch[f'ba_{item}']
        ab_unsup_value = unsup_batch[f'ab_{item}']

        # 谁短补谁，ba_unsup_value短
        if sup_max_len == cur_max_len:
            padding_value = torch.zeros((ba_unsup_value.size(0), cur_max_len - unsup_max_len),
                                        dtype=ba_unsup_value.dtype)
            ba_unsup_value = torch.cat([ba_unsup_value, padding_value], dim=-1)

        else:
            padding_value = torch.zeros((sup_value.size(0), cur_max_len - sup_max_len),
                                        dtype=sup_value.dtype)
            sup_value = torch.cat([sup_value, padding_value], dim=-1)
        
        # 把 sup_batch 和 ba 的 数据放在一起
        grad_value = torch.cat([sup_value, ba_unsup_value], dim=0)

        grad_data[item] = grad_value.to(config['device'])
        no_grad_data[item] = ab_unsup_value.to(config['device'])

    return grad_data, no_grad_data

In [41]:
def forward_no_grad(no_grad_data, config, model):
    with torch.no_grad():
        np_grad_logits = model(**no_grad_data)[0]
        # ----------- sharpen -------------#
        no_grad_probs = torch.softmax(np_grad_logits / config['uda_softmax_temp'], dim=-1)
        # ----------- sharpen -------------#
        # largest_probs [B] [0.879, 0.987, 0.234, 0.768, 0.333]
        largest_probs, _= no_grad_probs.max(dim=-1)
        unsup_loss_mask = largest_probs.gt(config['uda_confidence_threshold']).float()
        # unsup_loss_mask tensor([True, True, False, True, False])
    return unsup_loss_mask, no_grad_probs

In [42]:
def forward_with_grad(unsup_loss_mask, unsup_probs, config, cur_bs, 
    model, grad_data, total_steps, global_steps):
    # 得到\eta值， 随着训练的进行，阈值逐渐变大，最后是1，把所有监督数据都用上了
    tsa_threshold = get_tsa_threshold(total_steps, global_steps)
    
    logits = model(**grad_data)[0]
    # --------- 有监督损失 -------#
    # cur_bs 无监督 ba 的 batch_size
    # 前面一部分是 train 的 sup_data, 后面是unsup_data
    sup_logits, unsup_logits = logits.split([logits.size(0)-cur_bs, cur_bs])

    # 得到 sup_labels
    sup_labels = grad_data['labels'][:logits.size(0)-cur_bs]

    per_example_loss = nn.CrossEntropyLoss(reduction='none')(sup_logits, sup_labels)
    
    # 拿出 正确标签 对应的概率
    correct_label_probs = torch.softmax(sup_logits, dim=-1).gather(dim=-1, index=sup_labels.view(-1, 1))
    
    # 监督数据 过于自信不要，留下小于等于 tsa_threshold 的计算损失
    sup_loss_mask = correct_label_probs.le(tsa_threshold).squeeze().float()
    
    # 应用mask掩盖有监督数据过度自信的样本损失
    per_example_loss *= sup_loss_mask
    
    # 有效监督样本的平均损失
    sup_loss = per_example_loss.sum()/max(sup_loss_mask.sum(), 1) # max(sup_loss_mask.sum(), 1) 有效个数
    # --------- 有监督损失 -------#


    # --------- 无监督损失 -------#
    unsup_log_probs = torch.log_softmax(unsup_logits, dim=-1)
    # input 希望是一个对数概率
    # Target 目标为概率值
    per_example_kl_loss = nn.KLDivLoss(reduction='none')(unsup_log_probs, unsup_probs).sum(dim=-1)

    # 应用mask掩盖无监督数据中不自信的样本损失
    per_example_kl_loss *= unsup_loss_mask

    # 计算无监督样本的平均损失
    unsup_loss = per_example_kl_loss.sum()/max(unsup_loss_mask.sum(), 1)
    # --------- 无监督损失 -------#

    # 加权两种损失
    loss = sup_loss + unsup_loss

    # 多卡取平均
    if config['n_gpus']>1:
        loss = loss.mean()
        sup_loss = sup_loss.mean()
        unsup_loss = unsup_loss.mean()
    
    return loss, tsa_threshold, unsup_loss, sup_loss

In [43]:
from transformers import AdamW
from tqdm import trange
import os
def train(config, train_dataloader, dev_dataloader, unsup_dataloader=None):
    model = BertForAFQMC.from_pretrained(config['model_path'])

    optimizer = AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    model.to(config['device'])
    # unsup_dataloader train, dev, test
    # 使用 unsup_dataloader，因为unsup_dataloader比较大
    total_steps = len(unsup_dataloader) * config['num_epochs']
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.
    best_acc = 0.
    best_model_path = ''

    if config['n_gpus'] > 1:
        model = nn.DataParallel(model)

    train_iterator = iter(train_dataloader)
    for _ in epoch_iterator:
        unsup_iterator = tqdm(unsup_dataloader, desc='Training', total=len(unsup_dataloader))
        model.train()
        # ----------------------- new ----------------------#
        for unsup_batch in unsup_iterator:
            cur_bs = unsup_batch['ab_input_ids'].size(0)
            try:
                sup_batch = next(train_iterator)
            except StopIteration:
                train_iterator = iter(train_dataloader)
                sup_batch = next(train_iterator)
            
            # 返回 grad_data：需要计算梯度，需要进行反向传播的数据
            # 返回 no_grad_data: 不需要计算梯度，不需要进行反向传播的数据
            grad_data, no_grad_data = get_data(sup_batch, unsup_batch, config)
            
            # 无监督数据 (ab) 只需要正向传播
            # mask, ab_logits
            unsup_loss_mask, unsup_probs = forward_no_grad(no_grad_data, config, model)
             
            # 得出loss
            loss, tsa_threshold, unsup_loss, sup_loss = forward_with_grad(
                unsup_loss_mask, unsup_probs, config, cur_bs, model, grad_data, total_steps, global_steps
            )
            

            model.zero_grad()
            loss.backward()

            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            if config['ema_start']:
                ema.update()

            train_loss += loss.item()
            global_steps += 1

            unsup_iterator.set_postfix_str(f'running training loss: {loss.item():.4f}')
        
            if global_steps % config['logging_step'] == 0:
                if global_steps >= config['ema_start_step'] and not config['ema_start']:
                    print('\n>>> EMA starting ...')
                    config['ema_start'] = True
                    ema = EMA(model.module if hasattr(model, 'module') else model, decay=0.999)

                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss

                if config['ema_start']:
                    ema.apply_shadow()
                val_loss, f1, acc = evaluation(config, model, dev_dataloader)

                print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f}, '

                if acc > best_acc:
                    model_save_path = os.path.join(config['output_path'],
                                                   f'checkpoint-{global_steps}-{acc:.6f}')
                    model_to_save = model.module if hasattr(model, 'module') else model
                    model_to_save.save_pretrained(model_save_path)
                    best_acc = acc
                    best_model_path = model_save_path

                print_log += f'valid f1: {f1:.6f}, valid acc: {acc:.6f}'

                print(print_log)
                model.train()
                if config['ema_start']:
                    ema.restore()

    return model, best_model_path

In [44]:
model, best_model_path = train(config, train_dataloader, dev_dataloader, unsup_dataloader)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model were not used when initializing BertForAFQMC: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForAFQMC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForAFQMC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForAFQMC were not initialized from the model checkpoint at /content/drive/MyDrive/Cola


>>> training loss: 1.107390, valid loss: 0.610545, valid f1: 0.000000, valid acc: 0.689991



Training:  45%|████▌     | 799/1772 [09:11<10:22,  1.56it/s, running training loss: 1.1196]


>>> EMA starting ...




Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:40,  2.69it/s]

Evaluation:   1%|          | 2/270 [00:00<01:02,  4.30it/s]

Evaluation:   1%|          | 3/270 [00:00<00:48,  5.47it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:42,  6.30it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:37,  7.04it/s]

Evaluation:   2%|▏         | 6/270 [00:00<00:35,  7.50it/s]

Evaluation:   3%|▎         | 7/270 [00:01<00:32,  8.16it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:31,  8.22it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:32,  8.12it/s]

Evaluation:   4%|▎         | 10/270 [00:01<00:30,  8.44it/s]

Evaluation:   4%|▍         | 12/270 [00:01<00:26,  9.72it/s]

Evaluation:   5%|▍         | 13/270 [00:01<00:28,  9.13it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:29,  8.59it/s]

Evaluation:   6%|▌         | 15/270 [00:01<00:29,  8.66it/s]

Evaluation:   6%|▌         | 16/270 [00:02<00:30,  8.32it/s]

Evaluation:   6%|▋      

...


Training:  68%|██████▊   | 1199/1772 [14:15<05:40,  1.68it/s, running training loss: 0.9281]

Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:45,  2.56it/s]

Evaluation:   1%|          | 2/270 [00:00<01:02,  4.29it/s]

Evaluation:   1%|          | 3/270 [00:00<00:47,  5.60it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:41,  6.40it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:37,  7.07it/s]

Evaluation:   2%|▏         | 6/270 [00:00<00:35,  7.37it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:31,  8.31it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:32,  8.13it/s]

Evaluation:   4%|▍         | 11/270 [00:01<00:27,  9.28it/s]

Evaluation:   5%|▍         | 13/270 [00:01<00:27,  9.51it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:28,  9.02it/s]

Evaluation:   6%|▌         | 15/270 [00:01<00:28,  9.05it/s]

Evaluation:   6%|▌         | 16/270 [00:02<00:30,  8.31it/s]

Evaluation:   6%|▋         | 17/270 [00:02<00:31,  8.0


>>> training loss: 1.065879, valid loss: 0.570199, valid f1: 0.051209, valid acc: 0.690918



Training:  90%|█████████ | 1599/1772 [19:19<02:15,  1.27it/s, running training loss: 1.1881]

Evaluation:   0%|          | 0/270 [00:00<?, ?it/s]

Evaluation:   0%|          | 1/270 [00:00<01:44,  2.57it/s]

Evaluation:   1%|          | 2/270 [00:00<01:04,  4.17it/s]

Evaluation:   1%|          | 3/270 [00:00<00:50,  5.26it/s]

Evaluation:   1%|▏         | 4/270 [00:00<00:42,  6.22it/s]

Evaluation:   2%|▏         | 5/270 [00:00<00:38,  6.94it/s]

Evaluation:   2%|▏         | 6/270 [00:01<00:35,  7.36it/s]

Evaluation:   3%|▎         | 8/270 [00:01<00:30,  8.46it/s]

Evaluation:   3%|▎         | 9/270 [00:01<00:31,  8.37it/s]

Evaluation:   4%|▎         | 10/270 [00:01<00:30,  8.50it/s]

Evaluation:   4%|▍         | 12/270 [00:01<00:26,  9.60it/s]

Evaluation:   5%|▍         | 13/270 [00:01<00:28,  9.03it/s]

Evaluation:   5%|▌         | 14/270 [00:01<00:29,  8.55it/s]

Evaluation:   6%|▌         | 15/270 [00:01<00:29,  8.75it/s]

Evaluation:   6%|▌         | 16/270 [00:02<00:30,  8.3


>>> training loss: 0.999668, valid loss: 0.556800, valid f1: 0.310238, valid acc: 0.690918



100%|██████████| 1/1 [21:47<00:00, 1307.49s/it]


In [45]:
from zipfile import ZipFile
def predict(config, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Predicting', total=len(test_dataloader))
    test_preds = []

    model.eval()
    with torch.no_grad():
        for batch in test_iterator:
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            logits = model(**batch_cuda)[0]
            probs = torch.softmax(logits, dim=-1)
            test_preds.append(probs[:, 1].detach().cpu())
    

    test_preds = torch.cat(test_preds)
    test_preds = torch.stack(test_preds.split(2), dim=0).mean(dim=1).numpy()
    submission_path = os.path.join(config['output_path'], 'submission.tsv')
    test_df = pd.DataFrame(data={'prediction': test_preds})
    test_df.to_csv(submission_path, index=False, header=False, encoding='utf8', sep='\t')
    with ZipFile(os.path.join(config['output_path'], 'submission.zip'), 'w') as myzip:
        myzip.write(submission_path, 'submission.tsv')

In [46]:
predict(config, model, test_dataloader)

Predicting: 100%|██████████| 483/483 [00:47<00:00, 10.17it/s]
